If needed you can uncomment the line below and install fuzzywuzzy

!pip install fuzzywuzzy

In [4]:
import pandas as pd
from fuzzywuzzy import fuzz

A Basic example of fuzzy wuzzy. I generally find this is the best function to iterate over with fuzzywuzzy

In [6]:
fuzz.ratio('apple','apple pie')

71

# Basic Setup

In [9]:
#Lets start by importing a basic dataframe so that we can get an idea of how this work
import wrds
db=wrds.Connection(wrds_username=#######)
df=db.raw_sql('select conm, at from comp.funda where fyear>=2020').drop_duplicates().reset_index(drop=True)

Loading library list...
Done


In [20]:
df.head()

,conm,at
0,ADAMS DIVERSIFIED EQUITY FD,NaN
1,ARROW DOW JONES GLBL YLD ETF,NaN
2,AMERICAN AIRLINES GROUP INC,62008.000
3,CECO ENVIRONMENTAL CORP,419.314
4,INVESCO BOND FUND,NaN


In [23]:
len(df)

9521

Let's start by simplifying our names as much as possible. Fuzzymatching is accurate but is very computationally intensive.

At this point, I would recommend performing any additional filters that you can. Remove things with negatie assets for example.

In [26]:
df=df[df['at']>0].reset_index(drop=True)

Now, I would get the words in a consistent format and perform any additional filtes such as removing words or phrases. 

In [28]:
#lowercase strings
df['conm']=df['conm'].str.lower()

In [31]:
df.head()

,conm,at
0,american airlines group inc,62008.000
1,ceco environmental corp,419.314
2,pinnacle west capital corp,20020.421
3,prog holdings inc,1317.404
4,abbott laboratories,72548.000


In [34]:
#remove the phrase corp from the list
df['conm']=df['conm'].str.replace('corp','')

In [35]:
df.head()

,conm,at
0,american airlines group inc,62008.000
1,ceco environmental,419.314
2,pinnacle west capital,20020.421
3,prog holdings inc,1317.404
4,abbott laboratories,72548.000


# Method 1: Complete but slow

Now that we have applied our filters, we will begin to fuzzy match. This line of code will place all names in the data frame in lower case, and then only keep a unique value for each name

In [38]:
compnames=list(df['conm'].unique())

In [39]:
len(compnames)

6089

To fuzzy match at this point would require 6089^2 matches, 37075921 total iterations, so this would be very slow at this point. If your number of names is low, you can just run the code below

In [45]:
#This is a list of the 1st 200 and 2nd 200 names
lis1=compnames[:200]
lis2=compnames[201:400]

In [51]:
#For each item in both lists we will take the fuzzy between them
lis3=[]
for x in lis1:
    for y in lis2:
        lis4=[]
        lis4.append(x)
        lis4.append(y)
        lis4.append(fuzz.ratio(x,y))
        lis3.append(lis4)
new=pd.DataFrame(lis3)
new.columns=['name1','name2','fuzzy']

In [52]:
#The new dataframe lists out the fuzzies for each match
new.head()

,name1,name2,fuzzy
0,american airlines group inc,curtiss-wright,29
1,american airlines group inc,wendy's co,27
2,american airlines group inc,dana inc,40
3,american airlines group inc,danaher,34
4,american airlines group inc,data i/o,33


In [55]:
temp=new.groupby('name1')['fuzzy'].max().reset_index().rename(columns={'fuzzy':'max_fuzzy'})
temp.head()

,name1,max_fuzzy
0,abbott laboratories,47
1,abm industries inc,84
2,acme united,64
3,adams resources & energy inc,60
4,advanced micro devices,50


In [56]:
#merge the temporary data frame with the original
new=pd.merge(temp,new,on='name1')
#keep only the best fuzzy
new=new[new['max_fuzzy']==new['fuzzy']].reset_index(drop=True).drop('max_fuzzy',1)

In [63]:
#This is the best match for each name. You can see the importance of filtering things like "industry" or "energy" if not important
new.head()

,name1,name2,fuzzy
0,abbott laboratories,hmg courtland properties,47
1,abm industries inc,dycom industries inc,84
2,acme united,crawford united,64
3,adams resources & energy inc,dominion energy inc,60
4,adams resources & energy inc,franklin resources inc,60


# Method 2: Identifier match

The first method is practically infeasible for large data sets. A better method is to identify a necessary characteristic such as a zip code or state, then perform an outer merge. For example, here we will merge on assets

In [67]:
len(df)

6132

In [68]:
#Here we merge our two data frames with an outer merge on assets. As you can see, it increases the number of observations
df2=pd.merge(df,df.rename(columns={'conm':'conm2'}),on='at')

In [70]:
len(df2)

6200

In [71]:
#Because we have the same name on both sides, the fuzzies should be able to be perfect.
lis1=list(df2['conm'])
lis2=list(df2['conm2'])

In [73]:
lis3=[]
for x in range(0,len(lis1)):
    lis4=[]
    lis4.append(lis1[x])
    lis4.append(lis2[x])
    lis4.append(fuzz.ratio(lis1[x],lis2[x]))
    lis3.append(lis4)
new=pd.DataFrame(lis3)
new.columns=['name1','name2','fuzzy']

In [76]:
#as can be seen, some firms that had the same number of assets are matched together with low fuzzzies.
new.sort_values('fuzzy').head()

,name1,name2,fuzzy
1988,bakhu holdings,advanced proteome therapetcs,19
1986,advanced proteome therapetcs,bakhu holdings,19
124,southern natural gas co,cabot,21
123,cabot,southern natural gas co,21
766,union carbide,zebra technologies cp -cl a,24


At this point however, we can just use our technique from before

In [77]:
temp=new.groupby('name1')['fuzzy'].max().reset_index().rename(columns={'fuzzy':'max_fuzzy'})
temp.head()

,name1,max_fuzzy
0,01 communique laboratory inc,100
1,1-800-flowers.com,100
2,10x genomics inc,100
3,111 inc -adr,100
4,17 education & te -adr,100


In [78]:
#merge the temporary data frame with the original
new=pd.merge(temp,new,on='name1')
#keep only the best fuzzy
new=new[new['max_fuzzy']==new['fuzzy']].reset_index(drop=True).drop('max_fuzzy',1)

In [79]:
# Note now everything is a perfect match as expected
new.head()

,name1,name2,fuzzy
0,01 communique laboratory inc,01 communique laboratory inc,100
1,1-800-flowers.com,1-800-flowers.com,100
2,10x genomics inc,10x genomics inc,100
3,111 inc -adr,111 inc -adr,100
4,17 education & te -adr,17 education & te -adr,100


In [80]:
#note that now we have exactly the correct number of names as well.
len(new)

6132